ПРОЕКТ "Рекомендательные системы"

1. Загрузка датасетов

In [1]:
import pandas as pd
import numpy as np


from urllib.request import urlopen
!gdown  --id 1Jt2uOF40RvnXFKcLX-oP3yUMyhtIyJ-v
df_train = pd.read_csv(filepath_or_buffer='/content/train.csv', sep=',')
!gdown  --id 1d9OEzBfPpwnzMwvTiHuUnWFH-rxBbYSQ
df_test = pd.read_csv(filepath_or_buffer='/content/test.csv', sep=',')
!gdown  --id 1S0mr0E-di0gdmogkdJX42RnxcYLzYvVn
submission = pd.read_csv(filepath_or_buffer='/content/sample_submission.csv', sep=',')

Downloading...
From: https://drive.google.com/uc?id=1Jt2uOF40RvnXFKcLX-oP3yUMyhtIyJ-v
To: /content/train.csv
284MB [00:02, 110MB/s]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Downloading...
From: https://drive.google.com/uc?id=1d9OEzBfPpwnzMwvTiHuUnWFH-rxBbYSQ
To: /content/test.csv
27.2MB [00:00, 103MB/s] 


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Downloading...
From: https://drive.google.com/uc?id=1S0mr0E-di0gdmogkdJX42RnxcYLzYvVn
To: /content/sample_submission.csv
7.40MB [00:00, 45.0MB/s]


2. Подготовка к использованию пакета Surprise. Работает на пространстве меток [1,2,3,4,5] без внешней информации

In [2]:
!pip install surprise
from surprise import SVDpp, SVD
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate


     |████████████████████████████████| 6.5MB 2.7MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1675741 sha256=90063c6ae2f11617c2c24c677bfd0d14ab1415ff1e64eef44606800e2a24d8d5
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


3.Приведение датасетов к формату пакета Surprise "uid" - "iid" - "rating"

In [3]:
dataset = df_train[['userid', 'itemid', 'overall']]
dataset['overall'] = dataset['overall'].astype(int)
dataset.head()
dataset.overall.min(), dataset.overall.max()
recom = df_test[['userid', 'itemid']]
recom


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,userid,itemid
0,68877,7506
1,50442,15530
2,64349,15049
3,26206,11957
4,59366,12173
...,...,...
285960,294,25601
285961,119653,18204
285962,54673,2287
285963,16569,9343


4. Анализ данных на предмет "холодного старта"

In [ ]:
userid_dataset_unic = dataset.userid.unique()

userid_recom_unic = recom.userid.unique()


diff_userid = np.setdiff1d(userid_recom_unic,userid_dataset_unic)
diff_userid.shape # в тестовом наборе 48 уникальных юзеров из 109357, которых нет в обучающем датасете

(48,)

In [ ]:

recom.loc[recom['userid'].isin(userid_dataset_unic)].shape # из 285965 записей тестового набора 285712 
#                                                           составляют записи с user, которых нет в обучающем датасете

(285712, 2)

In [ ]:
itemid_dataset_unic = dataset.itemid.unique()

itemid_recom_unic = recom.itemid.unique()


diff_itemid = np.setdiff1d(itemid_recom_unic,itemid_dataset_unic)
diff_itemid.shape # в тестовом наборе 18 уникальных товаров из 37876, которых нет в обучающем датасете


(18,)

In [4]:
df = dataset
dt = recom
dt['ranking']=''
dt['ranking'] = dt.ranking.apply(lambda x:1)
dt

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,userid,itemid,ranking
0,68877,7506,1
1,50442,15530,1
2,64349,15049,1
3,26206,11957,1
4,59366,12173,1
...,...,...,...
285960,294,25601,1
285961,119653,18204,1
285962,54673,2287,1
285963,16569,9343,1


5. Обучение модели

In [5]:

#loading training data and test data
#train_data= pd.read_csv(r"C:\Users\Pucado R\Downloads\Data Science Nigeria\train.csv")
#test_data= pd.read_csv(r"C:\Users\Pucado R\Downloads\Data Science Nigeria\test.csv")

#data preparation
#prepareData= {'itemID': list(train_data.Joke_identifier),
#'userID': list(train_data.Viewers_ID),
#'rating': list(train_data.Rating)}

#testdata= {'itemID': list(test_data.Joke_identifier),
#'userID': list(test_data.Viewers_ID),
#'rating': list(test_data.Rating)}

#loading into dataframe
#df=pd.DataFrame(prepareData)
#dt=pd.DataFrame(testdata)

#reader class using only the rating parameter
reader= Reader(rating_scale=(1.0, 5.0))

#arranging dataframe
data= Dataset.load_from_df(df, reader)
testdat= Dataset.load_from_df(dt, reader)

#build full trainset and use cross validation for evaluation
trainset= data.build_full_trainset()

#Using the SVD algorithm.
algo = SVDpp()

#Train the algorithm on the trainset
model= algo.fit(trainset)






In [ ]:
dt.shape[0]

285965

6. Блок предсказания для тестового датасета

In [6]:
#predictions using uid and iid
#predictions = algo.predict(uid= 'A1', iid= 'Akpororo Lagos 1')
#predictions #this one works fine but I need to predict on the whole testdat

#predictions using dataset
#predictions= algo.predict(testdat)
out = []

for row in range(dt.shape[0]):
  predict = algo.predict(dt.userid[row], dt.itemid[row])
  out.append(predict)

7. Нормализация [1,2,3,4,5] -> [0,1] и выгрузка файла в Kaggle

In [7]:
pred = pd.DataFrame(out)
#pred.est = pred.est.apply(lambda x: 1 if x>=4 else 0)
#submission['rating'] = pred['est']
preds = pred.est
#submission.to_csv('ss.csv', index=None)

#submission
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()
submission['rating']= normalized_preds
submission.to_csv('ss.csv', index=False)
submission

,Id,rating
0,0,0.939599
1,1,0.909289
2,2,0.810206
3,3,0.709853
4,4,0.863456
...,...,...
285960,285960,0.987722
285961,285961,0.259582
285962,285962,0.670975
285963,285963,0.968138


In [8]:
from google.colab import files
files.download('ss.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

8. Результат в Kaggle 0.76789